In [7]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.


True

In [8]:
ee.Initialize(project='Phen-DWD')

EEException: Project 'projects/Phen-DWD' not found or deleted.